In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../data/train.csv')
counters = np.load('../data/value_counters.npy')

In [3]:
X_tr = train[train.columns[2:]]
y_tr = train.target

In [26]:
# Data augmentation
def augment_class(X):
    X_new = X.copy()

    for c in range(X.shape[1]):
        np.random.shuffle(X_new[:, c])

    return X_new

def augment(X, y, t=2, balancer=2):
    np.random.seed(42)

    t_pos = t
    t_neg = t // balancer

    X_pos_orig = X[y == 1]
    X_neg_orig = X[y == 0]
    X_pos = np.zeros((t_pos, *X_pos_orig.shape), dtype=X.dtype)
    X_neg = np.zeros((t_neg, *X_neg_orig.shape), dtype=X.dtype)

    for i in range(t_pos):
        X_pos[i] = augment_class(X_pos_orig)

    for i in range(t_neg):
        X_neg[i] = augment_class(X_neg_orig)

    X_pos = np.vstack(X_pos)
    X_neg = np.vstack(X_neg)
    y_pos = np.ones(X_pos.shape[0])
    y_neg = np.zeros(X_neg.shape[0])
    X = np.vstack((X, X_pos, X_neg))
    y = np.concatenate((y, y_pos, y_neg))

    return X, y

In [6]:
%load_ext line_profiler

In [18]:
X_neg = X_tr[y_tr == 0]

In [27]:
%lprun -f augment_class augment_class(X_neg.values)

In [28]:
%lprun -f augment augment(X_tr.values, y_tr.values, t=10, balancer=3)